<a href="https://colab.research.google.com/github/deseyeh/AML-Detection-IBM-ALM/blob/main/AML_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# initiate path

path = "/content/drive/My Drive/Colab Notebooks/AML_data/HI-Large_Patterns.txt"
print(path)

/content/drive/My Drive/Colab Notebooks/AML_data/HI-Large_Patterns.txt


### Load dataset

In [ ]:
try:
    data = pd.read_csv(path)
except IOError:
    print("File Not Found")

In [ ]:
data.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


### Shape

In [ ]:
num_rows, num_columns = data.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 5078345
Number of columns: 11


### Info

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 426.2+ MB


### Missing Value

In [ ]:
data.isna().sum()

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64

In [ ]:
print(f"Total missing value: {data.isna().sum().sum()}")

Total missing value: 0


### Duplicate value

In [ ]:
duplicate_rows = data.duplicated()

In [ ]:
duplicate_rows.sum()

9

In [ ]:
data[duplicate_rows]

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
863295,2022/09/01 16:20,12004,800C927C1,12004,800C927C0,0.000008,Bitcoin,0.080000,Euro,ACH,0
863296,2022/09/01 16:20,12004,800C927C1,220,813D8C1E1,0.000008,Bitcoin,0.000008,Bitcoin,Bitcoin,0
3684005,2022/09/07 21:25,29992,8099A29B1,220,813725AE1,0.000003,Bitcoin,0.000003,Bitcoin,Bitcoin,0
4166786,2022/09/08 21:05,113779,811144AB1,113779,811144AB0,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
4166787,2022/09/08 21:05,113779,811144AB1,53744,813C777F1,0.000002,Bitcoin,0.000002,Bitcoin,Bitcoin,0
4510480,2022/09/09 10:03,6075,80C702911,6075,80C702910,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
4510481,2022/09/09 10:03,6075,80C702911,154653,814389B61,0.000002,Bitcoin,0.000002,Bitcoin,Bitcoin,0
4816512,2022/09/09 21:33,14433,80935A891,14433,80935A890,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
4816513,2022/09/09 21:33,14433,80935A891,15,813F7AE61,0.000001,Bitcoin,0.000001,Bitcoin,Bitcoin,0


#### Remove duplicate

In [ ]:
# Remove duplicate rows and keep the first occurrence
data = data.drop_duplicates()

In [ ]:
data.shape

(5078336, 11)

In [ ]:
# initial datashape
''''
Number of rows: 5078345
Number of columns: 11
''''''

## Description of Data

In [ ]:
# the columns of dataset
data.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [ ]:
##
data_desc = data.describe()
data_desc

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
count,5.078336e+06,5.078336e+06,5.078336e+06,5.078336e+06,5.078336e+06
mean,4.573058e+04,6.574461e+04,5.988737e+06,4.509281e+06,1.019428e-03
std,8.176567e+04,8.409303e+04,1.037184e+09,8.697736e+08,3.191221e-02
min,1.000000e+00,1.000000e+00,1.000000e-06,1.000000e-06,0.000000e+00
25%,1.190000e+02,4.259000e+03,1.833700e+02,1.844800e+02,0.000000e+00
50%,9.679000e+03,2.156800e+04,1.411040e+03,1.414570e+03,0.000000e+00
75%,2.862800e+04,1.223320e+05,1.234630e+04,1.229815e+04,0.000000e+00
max,3.563030e+05,3.562940e+05,1.046302e+12,1.046302e+12,1.000000e+00


### Export data for Model further preprocessing

In [ ]:
#processed1.csv